#NAME ENTITY RECOGNITION: 
   1.Project of NLP

In [180]:
#Import data:

import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NAME ENTITY RECOGNITION PROJECT/Name_entity_recognition_dataset.csv')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [368]:
#check the values:

data['Tag'].value_counts()

O        64962
B-geo     2444
B-gpe     1407
B-org     1400
I-per     1398
B-tim     1370
B-per     1250
I-org     1081
I-geo      484
I-tim      384
B-art       63
I-gpe       49
B-eve       47
I-eve       40
I-art       35
B-nat       24
I-nat       10
Name: Tag, dtype: int64

#Dictionary Creation & Vocabulary:

In [182]:
#get the dictionaries:

def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}      
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok

In [183]:
#dictionaries are formed
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [184]:
tag2idx

{'B-art': 15,
 'B-eve': 10,
 'B-geo': 9,
 'B-gpe': 4,
 'B-nat': 3,
 'B-org': 11,
 'B-per': 14,
 'B-tim': 12,
 'I-art': 2,
 'I-eve': 17,
 'I-geo': 8,
 'I-gpe': 5,
 'I-nat': 7,
 'I-org': 16,
 'I-per': 13,
 'I-tim': 6,
 'O': 0,
 nan: 1}

In [255]:
tag2idx = {'B-art': 15,
 'B-eve': 10,
 'B-geo': 9,
 'B-gpe': 4,
 'B-nat': 3,
 'B-org': 11,
 'B-per': 14,
 'B-tim': 12,
 'I-art': 2,
 'I-eve': 17,
 'I-geo': 8,
 'I-gpe': 5,
 'I-nat': 7,
 'I-org': 16,
 'I-per': 13,
 'I-tim': 6,
 'O': 1,
}

In [258]:
idx2tag = {1 : 'O',
 2: 'I-art',
 3: 'B-nat',
 4: 'B-gpe',
 5: 'I-gpe',
 6: 'I-tim',
 7: 'I-nat',
 8: 'I-geo',
 9: 'B-geo',
 10: 'B-eve',
 11: 'B-org',
 12: 'B-tim',
 13: 'I-per',
 14: 'B-per',
 15: 'B-art',
 16: 'I-org',
 17: 'I-eve'}

In [257]:
# tag2idx & token2idx are formed:

In [259]:
#Lets map the idx of the word(tokens)
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)

In [260]:
data['Word_idx'][0]

3031

In [262]:
data.isnull().sum()

Sentence #    72982
Word              0
POS               1
Tag               1
Word_idx          0
Tag_idx           1
dtype: int64

In [263]:
#Let's fill the column by 'ffill' as Sentence0 should come as again in column:

data_fillna = data.fillna(method='ffill', axis=0)

In [264]:
#No null values:
data_fillna.isnull().sum()

Sentence #    0
Word          0
POS           0
Tag           0
Word_idx      0
Tag_idx       0
dtype: int64

#Important step to create a Proper Dataset:

In [191]:
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [267]:
data_group

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[3031, 1417, 1598, 875, 8815, 3610, 3343, 687,...","[0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0, 0, ..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[5721, 3524, 6759, 5527, 3798, 687, 5378, 5780...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[1488, 7322, 1670, 3308, 8769, 5440, 7611, 242...","[0, 0, 12, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 11..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[9355, 8212, 2404, 6404, 5977, 6899, 7650, 603...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Sentence: 1001,"[The, global, financial, crisis, has, left, Ic...","[DT, JJ, JJ, NN, VBZ, VBN, NNP, POS, NN, IN, N...","[O, O, O, O, O, O, B-org, O, O, O, O, O]","[3576, 9132, 9297, 2808, 1155, 8212, 8111, 939...","[0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...
3462,Sentence: 995,"[At, the, close, of, World, War, II, ,, German...","[IN, DT, NN, IN, NNP, NNP, NNP, ,, NNP, VBD, V...","[O, O, O, O, O, O, B-eve, O, B-geo, O, O, O, O...","[7390, 2428, 8005, 1417, 101, 6676, 6936, 6942...","[0, 0, 0, 0, 0, 0, 10, 0, 9, 0, 0, 0, 0, 0, 0,..."
3463,Sentence: 996,"[The, wall, was, built, starting, in, 1961, an...","[DT, NN, VBD, VBN, VBG, IN, CD, CC, VBD, TO, V...","[O, O, O, O, O, O, B-tim, O, O, O, O, O, O, O,...","[3576, 126, 6296, 3666, 3583, 7611, 6428, 1132...","[0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0,..."
3464,Sentence: 997,"[The, wall, was, breached, on, November, 9, ,,...","[DT, NN, VBD, VBN, IN, NNP, CD, ,, CD, IN, DT,...","[O, O, O, O, O, B-tim, I-tim, I-tim, I-tim, O,...","[3576, 126, 6296, 3467, 6866, 2145, 4406, 6942...","[0, 0, 0, 0, 0, 12, 6, 6, 6, 0, 0, 0, 0, 0, 0,..."
3465,Sentence: 998,"[Thousands, of, Icelanders, marked, the, 90th,...","[NNS, IN, NNS, VBD, DT, JJ, NN, IN, NN, IN, NN...","[O, O, B-gpe, O, O, O, O, O, O, O, B-org, B-ti...","[3031, 1417, 1311, 7887, 2428, 2836, 9107, 141...","[0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 0, 0..."


#Libraries

In [268]:
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [269]:
len(tag2idx)

17

#"Padding & Creating (Train, Test, Val Dataset)"

In [272]:
#Pad tokens (X var)    
tokens = data_group['Word_idx'].tolist()
maxlen = max([len(s) for s in tokens])
pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post')

#Pad Tags (y var) and convert it into one hot encoding
tags = data_group['Tag_idx'].tolist()
pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value=0)
n_tags = len(tag2idx)+1
pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

#Split train, test and validation set
tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)
    
    

In [276]:
train_tags[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [277]:
train_tokens[0]

array([7207, 7611, 8931, 6759, 3001, 1132, 8839, 3972,  875, 6423, 4298,
       4553, 9205, 7611, 6694, 7611, 2428, 2395, 9398, 1407, 9352, 9211,
       3372,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [130]:
#Final Output: Expected to 18 values:
len(train_tags[1][0])

18

In [278]:
len(train_tokens)

2340

In [279]:
len(test_tokens)

347

In [280]:
len(val_tokens)

780

In [281]:
tokens = data_group['Word_idx'].tolist()
max([len(s) for s in tokens])

70

In [282]:

import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout, Bidirectional


In [287]:
input_dim = len(list(set(data['Word'].to_list())))+1                   #9437
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])  #70
n_tags = len(tag2idx)+1


#Build the Model:

In [288]:
model = Sequential()

    # Add Embedding layer
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add bidirectional LSTM
model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2), merge_mode = 'concat'))

     # Add LSTM
model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.2))

    # Add dense layer
model.add((Dense(n_tags, activation="softmax")))

    # Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    

In [323]:
model.fit(train_tokens, np.array(train_tags),epochs=25,validation_data=(val_tokens,np.array(val_tags)))

Epoch 1/25
74/74 [==============================] - 10s 130ms/step - loss: 0.0648 - accuracy: 0.9800 - val_loss: 0.1112 - val_accuracy: 0.9708
Epoch 2/25
74/74 [==============================] - 10s 129ms/step - loss: 0.0601 - accuracy: 0.9819 - val_loss: 0.1114 - val_accuracy: 0.9703
Epoch 3/25
74/74 [==============================] - 9s 128ms/step - loss: 0.0570 - accuracy: 0.9831 - val_loss: 0.1162 - val_accuracy: 0.9719
Epoch 4/25
74/74 [==============================] - 10s 129ms/step - loss: 0.0531 - accuracy: 0.9846 - val_loss: 0.1115 - val_accuracy: 0.9715
Epoch 5/25
74/74 [==============================] - 10s 129ms/step - loss: 0.0492 - accuracy: 0.9859 - val_loss: 0.1117 - val_accuracy: 0.9722
Epoch 6/25
74/74 [==============================] - 9s 128ms/step - loss: 0.0456 - accuracy: 0.9872 - val_loss: 0.1097 - val_accuracy: 0.9730
Epoch 7/25
74/74 [==============================] - 10s 130ms/step - loss: 0.0417 - accuracy: 0.9885 - val_loss: 0.1085 - val_accuracy: 0.9749
E

In [378]:
x1 = test_tokens[7]

In [379]:
y1 = test_tags[7]

In [380]:
y_test = np.argmax(y1, axis=1)

In [381]:
y_test

array([0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [382]:
y_pred = model.predict(x1)

In [383]:
y = y_pred.reshape(70,18)

In [384]:
k=np.argmax(y,axis=1)

In [385]:
k

array([0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])